## Import required packages

In [0]:
# Download required packages
!pip -q install gdown missingno torch

%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark import SparkContext, SparkConf
from pyspark.sql.window import Window


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import missingno as msno
import torch
import torch.nn as nn
from typing import *
import datetime
import gdown

import tqdm as tq
def tqdm(*args, **kwargs):
  ''' Small trick to prevent tqdm printing newlines at each step. '''
  return tq.tqdm(*args, **kwargs, leave=True, position=0)

WARNING: You are using pip version 20.2.4; however, version 21.1.1 is available.
You should consider upgrading via the '/databricks/python3/bin/python -m pip install --upgrade pip' command.

## Data aquisition
We retrieve our datasets and download them to a temporary directory in the driver node.

In [0]:
!rm -rf /tmp/data /tmp/__MACOSX
gdown.download('https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM', '/tmp/data.zip', quiet=False)
!unzip -q /tmp/data.zip -d /tmp/
!rm /tmp/data.zip

Downloading...
From: https://drive.google.com/uc?id=1ggmDp-AWFzbQReLG0pLpQE_3fO0C0RnM
To: /tmp/data.zip
0.00B [00:00, ?B/s]8.91MB [00:00, 49.5MB/s]24.1MB [00:00, 94.1MB/s]35.1MB [00:00, 64.0MB/s]50.9MB [00:00, 87.3MB/s]68.7MB [00:00, 111MB/s] 71.9MB [00:00, 95.3MB/s]

Then we load the datasets to the DBFS.

In [0]:
dbutils.fs.mv("file:/tmp/data", "dbfs:/data", recurse=True)

Out[3]: True

In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


In [0]:
%fs ls /data/

path,name,size
dbfs:/data/.DS_Store,.DS_Store,6148
dbfs:/data/key_stats_yahoo.csv,key_stats_yahoo.csv,2047081
dbfs:/data/prices/,prices/,0


## Dataset loading

In [0]:
key_stats_df = spark.read.load("dbfs:/data/key_stats_yahoo.csv", 
                           format="csv",
                           sep=",",
                           inferSchema="true",
                           header="true"
                          )

# Drop the first ID column
key_stats_df = sc.parallelize(key_stats_df.drop(key_stats_df.columns[0]).head(1005)).toDF()#TODO: remove head(n) (only meant for development)
key_stats_df.schema['Date'].nullable = False

# Use legacy format to parse dates
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
key_stats_df = key_stats_df.withColumn("Date", F.to_date(key_stats_df["Date"], 'MM/dd/yyyy HH:mm'))

# Cast numerical columns to double
for column in key_stats_df.columns[2:]:
  key_stats_df = key_stats_df.withColumn(column, key_stats_df[column].cast("double"))

# Prices dataframes for every stock #TODO: remove :N (only meant for development)
prices_files = [f.path for f in dbutils.fs.ls('/data/prices/')[:10] if f.path.endswith('.csv')]
dfs_names = [f.rsplit('/', 1)[1][:-len('.csv')] for f in prices_files]
prices_dfs = []
for f in tqdm(prices_files, desc='Reading stock price data', total=len(prices_files)):
  df = spark.read.load(f,
                       format="csv",
                       sep=",",
                       inferSchema="true",
                       header="true"
                      )
  df = df.withColumn("Date", F.to_date(df["Date"], 'dd-MM-yyyy'))
  df.schema['Date'].nullable = False
  prices_dfs.append(df)

Reading stock price data: 0%| | 0/9 [00:00<?, ?it/s]Reading stock price data: 11%|█ | 1/9 [00:01<00:11, 1.42s/it]Reading stock price data: 22%|██▏ | 2/9 [00:03<00:11, 1.69s/it]Reading stock price data: 33%|███▎ | 3/9 [00:05<00:10, 1.80s/it]Reading stock price data: 44%|████▍ | 4/9 [00:07<00:10, 2.10s/it]Reading stock price data: 56%|█████▌ | 5/9 [00:09<00:07, 1.91s/it]Reading stock price data: 67%|██████▋ | 6/9 [00:10<00:05, 1.76s/it]Reading stock price data: 78%|███████▊ | 7/9 [00:12<00:03, 1.68s/it]Reading stock price data: 89%|████████▉ | 8/9 [00:13<00:01, 1.57s/it]Reading stock price data: 100%|██████████| 9/9 [00:14<00:00, 1.41s/it]Reading stock price data: 100%|██████████| 9/9 [00:14<00:00, 1.64s/it]

## Dataset analysis

In [0]:
print("Prices dataframe format:")
prices_dfs[0].printSchema()

Prices dataframe format:
root
-- Date: date (nullable = true)
-- Low: double (nullable = true)
-- Open: double (nullable = true)
-- Volume: integer (nullable = true)
-- High: double (nullable = true)
-- Close: double (nullable = true)
-- Adjusted Close: double (nullable = true)

In [0]:
print("Key stats dataframe format:")
key_stats_df.printSchema()

Key stats dataframe format:
root
-- Date: date (nullable = true)
-- Ticker: string (nullable = true)
-- Price: double (nullable = true)
-- DE Ratio: double (nullable = true)
-- Trailing P/E: double (nullable = true)
-- Price/Sales: double (nullable = true)
-- Price/Book: double (nullable = true)
-- Profit Margin: double (nullable = true)
-- Operating Margin: double (nullable = true)
-- Return on Assets: double (nullable = true)
-- Return on Equity: double (nullable = true)
-- Revenue Per Share: double (nullable = true)
-- Market Cap: double (nullable = true)
-- Enterprise Value: double (nullable = true)
-- Forward P/E: double (nullable = true)
-- PEG Ratio: double (nullable = true)
-- Enterprise Value/Revenue: double (nullable = true)
-- Enterprise Value/EBITDA: double (nullable = true)
-- Revenue: double (nullable = true)
-- Gross Profit: double (nullable = true)
-- EBITDA: double (nullable = true)
-- Net Income Avl to Common : double (nullable = true)
-- Diluted EPS: double (nullable = true)
-- Earnings Growth: double (nullable = true)
-- Revenue Growth: double (nullable = true)
-- Total Cash: double (nullable = true)
-- Total Cash Per Share: double (nullable = true)
-- Total Debt: double (nullable = true)
-- Current Ratio: double (nullable = true)
-- Book Value Per Share: double (nullable = true)
-- Cash Flow: double (nullable = true)
-- Beta: double (nullable = true)

### Utility functions

In [0]:
# TODO: add remaining utility functions

def prices_df_nan_summary(prices_dfs: List[pyspark.sql.DataFrame], names: List[str]) -> pd.DataFrame:
  ''' Utility function to summarize columns that have missing values. '''
  nan_dfs = []
  for prices_df, name in tqdm(zip(prices_dfs, names), total=len(prices_dfs), desc='Generating prices summary ...'):
    nan_absolute = prices_df.agg(*[F.count(F.when(F.isnull(c), c)).alias(c) for c in prices_df.columns]).first()
    if any(nan_absolute):
      # Simple conversion from Pyspark row -> Python set of values
      values = set(nan_absolute.asDict().values()).difference({0})
      # Either we don't have values for that row, or we have all of them (but Date which is non-nullable)
      # Values contains the no. of NaN values and 0 in correspondance of the Date column
      assert len(values) == 1
      nan_count = values.pop()
      nan_dfs.append((name, round(100*nan_count/prices_df.count(), 3), nan_count))

  return pd.DataFrame(nan_dfs, columns=['Stock name', 'Missing data (%)', 'Count'])

def remove_trailing_nan(df: pyspark.sql.DataFrame, ticker: str, col: str = 'Low') -> pyspark.sql.DataFrame:
  '''
    A trick to detect if the input DataFrame ends with a contiguous collection of NaN rows, returns the dataframe without them.
  '''
  # The total number of rows of the dataframe
  df_length = df.count()
  
  # Sort the input dataframe and add a new column to keep track of the relative position of each row
  df_sorted_id = df.sort('Date').withColumn('id', F.row_number().over(Window.orderBy('Date')))
  
  # Tricky part here: create a new column called 'cumsum' that will store the progressive number of consecutive NaN in our dataset.
  # Let's break it into smaller parts:
  # 1) create an index generator that will partition by 'Low' values [(...,Null...), (...,value1,...), (...,value2...), ... (...)] and within rows order by date
  # Example:
  ## |2019-06-05|null|null|  null|null| null|          null|624|        21|
  ## |2019-06-06|null|null|  null|null| null|          null|625|        22|
  ## |2019-06-07|null|null|  null|null| null|          null|626|        23| <- last column is the cumulative sum (i.e. the number of consecutive NaN)
  ## ...
  ## |2019-05-09|23.2|11.5|   7.3|4.2|  16.2|          29.1|1  |         0|
  ## |2019-05-10|23.2|11.5|   7.3|4.2|  16.2|          29.1|2  |         0|
  # 2) assign to each row a progressive index starting from 1 if it has null in correspondance of Low, zero otherwise
  # 3) store these values into a new column called cumsum (i.e. it behaves like pandas cumsum)
  # 4) at the end, the row whose ID corresponds to the length of the dataframe will contain at column 'cumsum' the no. of trailing NaN values.
  cumsum_df = df_sorted_id.withColumn('cumsum', F.when(F.isnull(df_sorted_id.Low), F.row_number().over(Window.partitionBy('Low').orderBy('Date'))).otherwise(0))

  # Retrieve the "last" row and read the value of cumsum
  end_idx = cumsum_df.where(cumsum_df['id'] == df_length).first().cumsum
  
  # Retain rows whose index is lower len(df) - end_idx + 1 (i.e. cut trailing NaN values)
  return df_sorted_id.where(df_sorted_id['id'] <= df_length-end_idx+1)

def merge_prices_fundamentals(
    prices_dfs: List[pyspark.sql.DataFrame],
    key_stats_df: pyspark.sql.DataFrame,
    dfs_names: List[str],
    drop_cols: List[str] = ['Date', 'Ticker', 'Price']
    ) -> List[pyspark.sql.DataFrame]:
  # Define the target list of dataframes
  prices_dfs_new = []
  for ticker in tqdm(key_stats_df.select('Ticker').distinct().collect(), desc='Merging the datasets ...'):
    ticker_df = key_stats_df.filter((F.col('Ticker') == ticker)).collect()
    pass
    
                
def fill_missing_days(aggregate_dfs: List[pyspark.sql.DataFrame], remove_weekends: bool = True, end_year: int = 2013) -> List[pyspark.sql.DataFrame]:
  pass

def missing_values_summary(df):
  ''' Returns a utility summary to view missing values in our dataframe. '''
  n = df.count()
  
  def to_percentage(x: pyspark.sql.column.Column, n: int) -> int:
    ''' Utility function to compute the amount of missing values as a percentage of the original dataframe. '''
    return F.round(100 * x / n, 3)
  
  # Aggregate using the count function over null values, and return a view over the obtained (single row) dataframe
  return df.agg(*[to_percentage(F.count(F.when(F.isnull(c), c)), n).alias(c) for c in df.columns]).first()

In [0]:
print("Overview of the missing values in the key_stats dataframe\n")
key_stats_summary = missing_values_summary(key_stats_df)
key_stats_summary

Overview of the missing values in the key_stats dataframe

Out[8]: Row(Date=0.0, Ticker=0.0, Price=1.99, DE Ratio=18.607, Trailing P/E=9.652, Price/Sales=0.498, Price/Book=2.189, Profit Margin=1.194, Operating Margin=0.498, Return on Assets=1.99, Return on Equity=2.786, Revenue Per Share=0.398, Market Cap=0.199, Enterprise Value=1.095, Forward P/E=46.269, PEG Ratio=4.776, Enterprise Value/Revenue=1.294, Enterprise Value/EBITDA=7.96, Revenue=1.294, Gross Profit=2.886, EBITDA=7.662, Net Income Avl to Common =0.1, Diluted EPS=0.995, Earnings Growth=17.015, Revenue Growth=1.194, Total Cash=2.189, Total Cash Per Share=1.095, Total Debt=8.159, Current Ratio=6.567, Book Value Per Share=0.199, Cash Flow=11.542, Beta=2.886)

### Missing values imputation

In [0]:
summary = prices_df_nan_summary(prices_dfs, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset before preprocessing (only columns with missing values are displayed)")

Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:01<00:11, 1.46s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:02<00:10, 1.48s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:04<00:07, 1.33s/it]Generating prices summary ...: 44%|████▍ | 4/9 [00:06<00:08, 1.60s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:06<00:05, 1.33s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:07<00:03, 1.15s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:08<00:02, 1.08s/it]Generating prices summary ...: 89%|████████▉ | 8/9 [00:09<00:00, 1.00it/s]Generating prices summary ...: 100%|██████████| 9/9 [00:10<00:00, 1.06it/s]Generating prices summary ...: 100%|██████████| 9/9 [00:10<00:00, 1.15s/it]
Out[9]:

For most of the above stocks with missing values, we noticed that they indeed exist up to a given time and after that no more data is available. It may due to a business failure, hence no more stocks will be exchanged from that moment on.

In [0]:
# Clear our input data from training NaN values
prices_dfs_new = [remove_trailing_nan(df,name) for df,name in tqdm(zip(prices_dfs, dfs_names), total=len(prices_dfs), desc='Removing trailing NaN values ...')]

# Remove INTH stock from our dataset since it contains many inactivity periods
#inth_idx = dfs_names.index('INTH') #TODO: uncomment
#del dfs_names[inth_idx] #TODO: uncomment
#del prices_dfs_new[inth_idx] #TODO: uncomment

summary = prices_df_nan_summary(prices_dfs_new, dfs_names)
px.bar(summary, x='Stock name', y='Missing data (%)', hover_data=['Count'], title="Stock price dataset after preprocessing (only columns with missing values are displayed)")

Removing trailing NaN values ...: 0%| | 0/9 [00:00<?, ?it/s]Removing trailing NaN values ...: 11%|█ | 1/9 [00:03<00:27, 3.47s/it]Removing trailing NaN values ...: 22%|██▏ | 2/9 [00:05<00:20, 2.87s/it]Removing trailing NaN values ...: 33%|███▎ | 3/9 [00:08<00:17, 2.93s/it]Removing trailing NaN values ...: 44%|████▍ | 4/9 [00:11<00:14, 2.80s/it]Removing trailing NaN values ...: 56%|█████▌ | 5/9 [00:13<00:09, 2.46s/it]Removing trailing NaN values ...: 67%|██████▋ | 6/9 [00:15<00:06, 2.25s/it]Removing trailing NaN values ...: 78%|███████▊ | 7/9 [00:16<00:04, 2.08s/it]Removing trailing NaN values ...: 89%|████████▉ | 8/9 [00:18<00:02, 2.05s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:20<00:00, 1.96s/it]Removing trailing NaN values ...: 100%|██████████| 9/9 [00:20<00:00, 2.30s/it]
Generating prices summary ...: 0%| | 0/9 [00:00<?, ?it/s]Generating prices summary ...: 11%|█ | 1/9 [00:02<00:17, 2.24s/it]Generating prices summary ...: 22%|██▏ | 2/9 [00:03<00:12, 1.74s/it]Generating prices summary ...: 33%|███▎ | 3/9 [00:05<00:11, 1.89s/it]Generating prices summary ...: 44%|████▍ | 4/9 [00:08<00:11, 2.28s/it]Generating prices summary ...: 56%|█████▌ | 5/9 [00:09<00:07, 1.92s/it]Generating prices summary ...: 67%|██████▋ | 6/9 [00:11<00:05, 1.77s/it]Generating prices summary ...: 78%|███████▊ | 7/9 [00:12<00:03, 1.71s/it]Generating prices summary ...: 89%|████████▉ | 8/9 [00:15<00:01, 1.89s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:16<00:00, 1.75s/it]Generating prices summary ...: 100%|██████████| 9/9 [00:16<00:00, 1.85s/it]
Out[10]:

At this point we use the fast forward imputation technique to fill-in missing values. Please note that in this case missing values are mostly due to holidays or periods when stocks are not exchanged.

### Building our new dataset

In [0]:
# Impute missing values in the prices dataset (i.e. fast-forward last valid values)

# define the window
window = Window.orderBy('Date')\
               .rowsBetween(Window.unboundedPreceding, 0)

for i in range(len(prices_dfs_new)):
  for col_name in prices_dfs_new[i].schema.names:
    col = F.last(prices_dfs_new[i][col_name], ignorenulls=True).over(window)
    prices_dfs_new[i] = prices_dfs_new[i].withColumn(col_name, col)

# In this case this dataframe contains financial reports that may contain NaN values either because that
# metric was not available at that time OR because it was monitoring an initial stage of a company growth.
# What we do is to apply the classic fast-forward, and fill initial missing values with zeroes.
# Please note: we also discard the 'Forward P/E' column since the imputation here would introduce too much noise.
key_stats_df_new = key_stats_df.drop('Forward P/E')
for col_name in key_stats_df_new.schema.names:
    col = F.last(key_stats_df_new[col_name], ignorenulls=True).over(window)
    key_stats_df_new = key_stats_df_new.withColumn(col_name, col)
key_stats_df_new = key_stats_df_new.fillna(0.)


# Before merging the two datasets we need to sort them
#prices_dfs_new = [df.orderBy('Date').reset_index(drop=True) for df in prices_dfs_new]
#key_stats_df_new = key_stats_df_new.sort_values(by=['Date']).reset_index(drop=True)

# Merge the stock price dataset with fundamental data of the relative company
aggregate_dfs = merge_prices_fundamentals(prices_dfs_new, key_stats_df_new, dfs_names)
                  
# TODO: add more functions

Merging the datasets ...: 0%| | 0/56 [00:00<?, ?it/s]Merging the datasets ...: 0%| | 0/56 [00:00<?, ?it/s]

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-3715022015688392> in <module> 
 26 
 27 # Merge the stock price dataset with fundamental data of the relative company 
 ---> 28 aggregate_dfs = merge_prices_fundamentals ( prices_dfs_new , key_stats_df_new , dfs_names ) 
 29 
 30 # TODO: add more functions 

 <command-3715022015688384> in merge_prices_fundamentals (prices_dfs, key_stats_df, dfs_names, drop_cols) 
 57 prices_dfs_new = [ ] 
 58 for ticker in tqdm ( key_stats_df . select ( 'Ticker' ) . distinct ( ) . collect ( ) , desc = 'Merging the datasets ...' ) : 
 ---> 59 ticker_df = key_stats_df . filter ( ( F . col ( 'Ticker' ) == ticker ) ) . collect ( ) 
 60 pass 
 61 

 /databricks/spark/python/pyspark/sql/column.py in _ (self, other) 
 110 def _ ( self , other ) : 
 111 jc = other . _jc if isinstance ( other , Column ) else other
 --> 112 njc = getattr ( self . _jc , name ) ( jc ) 
 113 return Column ( njc ) 
 114 _ . __doc__ = doc

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o3629.equalTo.
: java.lang.RuntimeException: Unsupported literal type class java.util.ArrayList [amd]
	at org.apache.spark.sql.catalyst.expressions.Literal$.apply(literals.scala:91)
	at org.apache.spark.sql.catalyst.expressions.Literal$.$anonfun$create$2(literals.scala:178)
	at scala.util.Failure.getOrElse(Try.scala:222)
	at org.apache.spark.sql.catalyst.expressions.Literal$.create(literals.scala:178)
	at org.apache.spark.sql.functions$.typedLit(functions.scala:134)
	at org.apache.spark.sql.functions$.lit(functions.scala:117)
	at org.apache.spark.sql.Column.$eq$eq$eq(Column.scala:285)
	at org.apache.spark.sql.Column.equalTo(Column.scala:308)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [0]:
key_stats_df_new